## 文本预处理

In [4]:
import nltk
import re
import string
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also']
# 分词
def tokenize_text(text):
    tokens=nltk.word_tokenize(text)#分词
    tokens=[token.strip() for token in tokens]#去除单词前后的空格或换行字符
    return tokens
# 移除特殊字符
def remove_special_characters(text):
    tokens=tokenize_text(text)
    pattern=re.compile('[{}]'.format(re.escape(string.punctuation)))
    #re.compile函数根据包含的正则表达式的字符串创建模式对象,可以实现更有效率的匹配.
    #re.escape(pattern) 可以对字符串中所有可能被解释为正则运算符的字符进行转义的应用函数
    #string.punctuation找出字符串中的所有的标点
    filtered_tokens=filter(None,[pattern.sub(' ',token) for token in tokens])
    # filter过滤掉不符合条件的元素.pattern.sub实现比普通字符串replace 更加强大的替换功能
    filtered_text=' '.join(filtered_tokens)
    return filtered_text
# 移除停用词
def remove_stopwords(text):
    tokens=tokenize_text(text)
    filtered_tokens=[token for token in tokens
                    if token not in stopword_list]
    filtered_text=' '.join(filtered_tokens)
    return filtered_text
# keep text characters
def keep_text_characters(text):
    filtered_tokens=[]
    tokens=tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]',token):
            filtered_tokens.append(token)
    filtered_text=' '.join(filtered_tokens)
    return filtered_text

## LSH

In [5]:
#coding:utf-8
import os
import random
import math
import sys
 
def to_unicode_or_bust(obj,encoding='utf-8'):
    if isinstance(obj, basestring):
        if not isinstance(obj, unicode):
            obj = unicode(obj, encoding)
    return obj
 
"""
此函数用于获取dir文件夹中的文件的内容
"""
def getFilesName(dir):
    fileList=[]
    t=os.walk(dir)
    file=dir+'\\'
    for item in t:
        for name in item[2]:
            fileList.append(file+name)
    return fileList
 
"""
此函数用于获得fileName文件中的内容，文件内容存放在字符串中返回
"""
def getFileContent(fileName):
#     file=open(fileName,"r")
#     fileContent=file.read()
#     fileContent=fileContent.replace("\t"," ")
#     fileContent=fileContent.replace("\n"," ")
#     fileContent=fileContent.replace("\r"," ")
#     file.close()
#     print(fileContent)
#     print('*'*50)
    with open(fileName,"r") as f:
        fileContent=f.read()
    return fileContent
#return fileConten.decode("gb2312").encode("utf-8")用于显示
 
"""
此函数用于对各个文件中的内容进行k-shingle，然后对词条进行哈希（此处就用字典存储了）
其中dir是文件夹的名称字符串类型，k是int型
"""
from nltk.util import ngrams
def getShingleList(dir,k):
    fileList=getFilesName(dir)
    shingleList=list()
    for fileName in fileList:
        fileContent=getFileContent(fileName)
        text=fileContent
        text=remove_special_characters(text)
        text=remove_stopwords(text)
        text=keep_text_characters(text)
        text=tokenize_text(text)
        text=list(ngrams(text, k))
        shingle = set()
        for t in text:
            shingle.add(' '.join(list(t)))       
#         for index in range(0,len(fileContent)-k+1):
#             shingle.add(fileContent[index:index+k])
#         print(shingle)
#         print('***'*50)
        
        
#         print(shingle)
#         print('#'*100)
        shingleList.append(shingle)
#     print(shingleList)
#     print('='*100)
    return shingleList
 
"""
此处是新版的函数，将哈希签名的矩阵换的行列换了一下，便于接下来使用
"""
def getMinHashSignature(shingleList,signatureNum):
    #tatalSet用于存放所有集合的并集
    totalSet=shingleList[0]
    for i in range(1,len(shingleList)):
        totalSet=totalSet|shingleList[i]
 
    temp=int(math.sqrt(signatureNum))
    #randomArray用于模拟随机哈希函数
    randomArray=[]
    #signatureList用于存放总的哈希签名
    signatureList=[]
    maxNum=sys.maxsize
    for i in range(signatureNum):
        randomArray.append(random.randint(1,temp*2))
        randomArray.append(random.randint(1,temp*2))
    #buketNum用于记录所有元素的个数，作为随机哈希函数的桶号
    buketNum=len(totalSet)
    """
    此处将不同文档的自己的哈希签名存成一个list，然后再进行汇总到一个总的list
    """
    for shingleSet in shingleList:
        """
        signature用于存放哈希函数产生的签名
        """
        signature=[]
        for i in range(signatureNum):
            minHash=maxNum
            for index,item in enumerate(totalSet):
                if item in shingleSet:
                    num=(randomArray[i*2]*index+randomArray[i*2+1])%buketNum
                    minHash=min(minHash,num)
            signature.append(minHash)
        signatureList.append(signature)
    return signatureList
 
"""
在使用局部敏感哈希中，我们假设行条(也叫分组)和每个行条中行的个数的乘积刚好等于总的签名数，这样可以减少不必要的繁琐，
这就要求我们在选取相关参数时要注意
"""
def localSensitiveHash(signatureList,filesName,signatureNum,bands):
    lshResult=[]
    """
    此循环用于初始化这个list，构造一个下三角的模拟数组，用于存放两个文件之间行的相似个数
    """
    for i in range(len(signatureList)):
        temp=[]
        for j in range(i):
            temp.append(0)
        lshResult.append(temp)
 
    row=int(signatureNum/bands)
    #此循环是对签名的行条进行处理
    for i in range(0,bands):
        dicResult={}
        index=i*row
        #此循环是对行条中的每一列进行计算，num用于记录签名的下标
        for num,signature in enumerate(signatureList):
            #hashCode用于记录每个行条的hash桶号
            hashCode = 0
            #遍历行条中的列，此处的hash函数就暂时用累加，稍后修改
            for j in range(index,index+row):
                hashCode+=signature[j]
            if hashCode in dicResult:
                dicResult[hashCode].append(num)
            else:
                dicResult[hashCode]=[num]
        for valueList in dicResult.values():
            #print valueList
            if(len(valueList)>=2):
                for index1 in range(len(valueList)):
                    for index2 in range(index1+1,len(valueList)):
                        lshResult[valueList[index2]][valueList[index1]]+=1
 
    similarityResult=[]
    for i in range(1,len(lshResult)):
        for j in range(len(lshResult[i])):
            similarity=lshResult[i][j]/(bands*1.0)
            similarityResult.append((similarity,filesName[i],filesName[j]))
    return similarityResult
 

## 调用计算

In [7]:
dir="./bbc/test"            # 存放文档的目录路径
filesName=getFilesName(dir)
shingleList=getShingleList(dir,2)
signatureList=getMinHashSignature(shingleList,200)
result=localSensitiveHash(signatureList,filesName,200,40)
result.sort(reverse=True)
for each in result:
    print(each)

(1.0, './bbc/test\\011.txt', './bbc/test\\001.txt')
(0.025, './bbc/test\\011.txt', './bbc/test\\010.txt')
(0.025, './bbc/test\\011.txt', './bbc/test\\007.txt')
(0.025, './bbc/test\\011.txt', './bbc/test\\004.txt')
(0.025, './bbc/test\\011.txt', './bbc/test\\002.txt')
(0.025, './bbc/test\\010.txt', './bbc/test\\001.txt')
(0.025, './bbc/test\\009.txt', './bbc/test\\007.txt')
(0.025, './bbc/test\\007.txt', './bbc/test\\005.txt')
(0.025, './bbc/test\\007.txt', './bbc/test\\001.txt')
(0.025, './bbc/test\\005.txt', './bbc/test\\004.txt')
(0.025, './bbc/test\\004.txt', './bbc/test\\002.txt')
(0.025, './bbc/test\\004.txt', './bbc/test\\001.txt')
(0.025, './bbc/test\\002.txt', './bbc/test\\001.txt')
(0.0, './bbc/test\\011.txt', './bbc/test\\009.txt')
(0.0, './bbc/test\\011.txt', './bbc/test\\008.txt')
(0.0, './bbc/test\\011.txt', './bbc/test\\006.txt')
(0.0, './bbc/test\\011.txt', './bbc/test\\005.txt')
(0.0, './bbc/test\\011.txt', './bbc/test\\003.txt')
(0.0, './bbc/test\\010.txt', './bbc/test